# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Изучение данных из файла

В самом начале импортируем все необходимые библиотеки

In [1]:
import pandas as pd


from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report


In [2]:
df =  pd.read_csv('/datasets/users_behavior.csv') # чтение данных их файла

In [3]:
display(df.head()) #просмотр первых 5 строк

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


На первый взгляд, ошибок в данных не видно. Посмотрим информацию о файле методом info()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В файле 5 столбцов. Тип данных float64 в четырех из них, и в одном тип - int64. Необходимости менять тип данных в столбах нет. Во всех столбцах одинаковое количество данных, следовательно пропусков нет.

В соответствии с документацией в файле есть следующие данные:

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Проверим, нет ли полностью дублирующихся строк:

df.duplicated().sum()

In [5]:
df.duplicated().sum()

0

Дубликатов не обнаружено.

**Вывод**


Предварительный анализ показал, что данные пригодны для дальнейшей работы с ними. Необходимости проводить предобработку нет: наименования столбцов написаны в нижнем регистре, нет пропусков в столбцах и полных дубликатов строк, тип данных верный.

Поэтому приступим непосредственно к решению поставленной задачи.

## Разбеение данных на выборки

Для начала нам необходимо разделить данные на три части: на обучающую, валидационную и тестовую выборки. Обучающая выборка будет самой большой и вместит 60% данных. С помощью нее мы будем обучать модель. Вторая выборка валидационная (20% от объема данных), нужна нам для проверки качества работы алгоритма во время обучения модели. И последняя третья выборка - тестовая. Ее размер 20% от общего объема данных. На ней мы будем проверять качество обученной модели.

Для начала извлечем из общего набора 40% данных, которые потом поделим попалам и одна часть будет у нас валидационной выборкой, а вторая тестовой.

In [6]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345) # выделение 40% от данных

In [7]:
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345) # разделение выборки попалам

В нашем распоряжении обучающий набор данных и целевой признак, который нужно предсказать по остальным признакам. Целевой признак у нас категориальный, поэтому нам предстоит решить задачу классификации. Для начала выделим в каждой из выборок признаки (features) и целевой признак (target).

In [8]:
features_train = df_train.drop(['is_ultra'], axis=1) # извлечение признаков 
target_train = df_train['is_ultra'] # извлечение целевого признака

In [9]:
display(features_train.shape)
display(target_train.shape)

(1928, 4)

(1928,)

In [10]:
features_valid = df_valid.drop(['is_ultra'], axis=1) # извлечение признаков 
target_valid = df_valid['is_ultra'] # извлечение целевого признака

In [11]:
display(features_valid.shape)
display(target_valid.shape)

(643, 4)

(643,)

In [12]:
features_test = df_test.drop(['is_ultra'], axis=1) # извлечение признаков 
target_test = df_test['is_ultra'] # извлечение целевого признака

In [13]:
display(features_test.shape)
display(target_test.shape)

(643, 4)

(643,)

Данные поделили, признаки и целевые признаки выделили. Приступим к обучению модели.

## Исследование качества разных моделей

### Решающее дерево

DecisionTreeClassifier - это структура для классификации деревом решений. В переменной model будет хранится наша модель. А чтобы запустить обучение вызовем метод fit(), которому передадим наши признаки и целевой признак. После обучения, вызовем метод predict() и передадим ему валидационную выборку для проверки работоспособности модели.

Поскольку от глубины дерева, будет зависеть как обучится модель, то попробуем в цикле подобрать оптимальную глубину дерева. Глубина дерева задается параметром max_depth. Оценивать, как обучилась модель будем по такой меткике качества, как accuracy - доля правильных ответов. Чем выше этот показатель, тем лучше обучилась модель.

In [14]:
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    
    
    
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))
   

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7744945567651633


Итак лучший результат показывает дерево с глубиной max_depth = 3.

### Случайный лес

Еще один алгоритм - случайный лес. Алгоритм обучает большое количество независимых друг от друга деревьев, а потом принимает решение на основе голосования. Алгоритм случайного леса - RandomForestClassifier. Чтобы управлять количеством деревьев, есть параметр n_estimators. Чем больше деревьев, тем лучше результат, но модель будет учится дольше.

In [15]:
best_model = None
best_result = 0
for est in range(1, 25):
    for depth in range(1, 13):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) # 
        model.fit(features_train, target_train) # обучение модели на тренировочной выборке
        model.predict(features_valid) 
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model = model
            best_result = result #  сохранение наилучшего значения метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result,';' "Число деревьев:", est, ';', "Глубина деревьев", depth)

Accuracy наилучшей модели на валидационной выборке: 0.8055987558320373 ;Число деревьев: 24 ; Глубина деревьев 12


Лучший результат показал лес из 24 деревьев. Я попробовала поставить большее количество деревьев, но accuracy не увеличивался (случайно запустила 224 дерева: accuracy такой же как и на 24 деревьях, но расчет шел значительно дольше).

### Логистическая регрессия

Последним посмотрим такой алгоритм, как логистическую регрессию (LogisticRegression). Эта модель хороша, тем, что вероятность переобучения не велика, но при этом качество обучения у нее хуже, чем у случайного леса.

In [16]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train) # обучение модели на тренировочной выборке
predictions_valid = model.predict(features_valid)
result = model.score(features_valid, target_valid) # получение метрики качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7107309486780715


Результат хуже, чем у решающего дерева и случайного леса: всего 71% правильных решений.

**Вывод**

В этом разделе мы обучили различные модели и протестировали их на валидационной выборке. Мы меняли гиперпараметры и смотрели как меняется величина accuracy. Accuracy — это показатель, который описывает общую точность предсказания модели, т.е. с какой точностью модель делает верный прогноз. При обучении модели решающим деревом, лушую accuracy дает дерево с глубиной max_depth = 3. В случае со случайным лесом лучший результат при n_estimators =24 и глубиной дерева = 12. Показатель accuracy в логистической регрессии самый низкий, всего 71%. Поскольку выборка у нас не большая, все модели обучались достаточно быстро. На большей выборке, расчет с помощью случайного леса, мог затянутся. 

В итоге мы выбрали для дальнейшей работы модель созданную с помощью случайного леса с количеством деревьев равному 24. В этом случае наш accuracy на валидационной выборке равен 80,56%.

## Проверка качества модели на тестовой выборке

Для того чтобы проверить работоспособность уже обученной модели, проверим ее на тестовой выборке. Если модель обучилась достаточно хорошо и нет переобучения и недообучения, то показатель accuracy на тестовой выборке будет близок к показателю accuracy на валидационной выборке.

In [17]:
model = RandomForestClassifier(random_state=12345, n_estimators=24, max_depth=12) 
model.fit(features_train, target_train) # обучение модели с помощью случайного леса из 24 деревьев
predictions_valid = model.predict(features_test)
result = model.score(features_test, target_test) # проверка модели на тестовой выборке
display("Accuracy наилучшей модели на тестовой выборке:", result)

'Accuracy наилучшей модели на тестовой выборке:'

0.80248833592535

Доля правильных ответов чуть ниже, но достаточно близка к значению полученному на валидационной выборке.

**Вывод**

Поведение обученной модели на тестовой выборке достаточно хорошее. Показатель accuracy , близок по значению accuracy полученному на валидационной выборке. 

## (бонус) Проверка модели на адекватность

Когда мы имеем дело с равными классами, то accuracy покажет нам достаточно адекватную картину. В случае с неравномерными классами эта метрика качества достаточно бесполезна. 

In [18]:
display(len(df.loc[df['is_ultra']==1]))

985

In [19]:
display(len(df.loc[df['is_ultra']==0]))

2229

В нашей первоначальной таблице количество строк с тариформ Смарт значительно больше, чем с количеством тарифа Ульта. А это говорит о том, что классы (нули - тариф Смарт и единицы -Тариф Ультра), распределены не равномерно.

Для оценки качества таких моделей используют другие метрики качества: precision и recall. Precision (точность) показывает, какова доля объектов, для которых модель предсказала ответ "1", дествительно имеют ответ 1. А recall показывает, какую часть объектов имеющих значение "1", выделила модель. Precision и recall не зависят, в отличие от accuracy, от соотношения классов и потому применимы в условиях несбалансированных выборок.

Когда встает вопрос, какую из этих метрик выбрать, есть компромис в виде агрегированного критерия качества -F-мера. F-мера достигает максимума при полноте и точности, равными единице, и близка к нулю, если один из аргументов близок к нулю.

В sklearn есть удобная функция metrics.classifical_report, которая одновременно расчитывает precision, recall и F-меру для каждого класса, а также выводит количество каждого класса.

Для того, чтобы проверить модель на вменяемость, нужно оценить ее адекватность на случайной выборке. В качестве случайной мы возьмем тестовую выборку и расчитаем метрики качества, с помощью metrics.classifical_report, для трех лучших моделей (по одной, из каждого вида).

Первым мы посмотрим решающее дерево с глубиной max_depth = 3.

In [20]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)

    
print(classification_report(target_test, predictions_test))

              precision    recall  f1-score   support

           0       0.79      0.93      0.85       440
           1       0.74      0.46      0.57       203

    accuracy                           0.78       643
   macro avg       0.77      0.69      0.71       643
weighted avg       0.77      0.78      0.76       643



Итак в 74% случаев модель правильно предсказывает, тариф Ультра, как указанный тариф, остальные 26% объектов, которые являются тарифом Ультра, таковым не являются. С тарифом Смарт, ситуация чуть получше, модель определяет верно 79%. Посмотрев на F-меру считаю, что модель не плохо справляется с определением тарифа Смарт, а вот тариф Ультра пока ей дается не очень хорошо. classification_report также дает нам информацию по средней оценке (macro-averaging) и средневзвешенной оценке (weighted average) precision, recall и F-меры. Средняя и средневзвешанная оценка precision ниже на 1%, расчитанного ранее значения accuracy. 

Далее посмотрим, как себя ведет модель созданная с помощью случайного леса, с количеством деревьев равным 24

In [21]:
model = RandomForestClassifier(random_state=12345, n_estimators=24, max_depth=12) 
model.fit(features_train, target_train) # обучение модели с помощью случайного леса из 24 деревьев
predictions_test = model.predict(features_test)
 
print(classification_report(target_test, predictions_test))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86       440
           1       0.74      0.58      0.65       203

    accuracy                           0.80       643
   macro avg       0.78      0.74      0.76       643
weighted avg       0.80      0.80      0.79       643



Только в 74% случаях модель правильно предсказывает, тариф Ультра, как указанный тариф, остальные 26% объектов, которые являются тарифом Ультра, таковым не являются. С тарифом Смарт, ситуация чуть получше, модель определяет верно 82%. Посмотрев на F-меру считаю, что модель не плохо справляется с определением тарифа Смарт, а вот тариф Ультра пока ей дается не очень хорошо.  Средняя и средневзвешанная оценка precision примерно равна, расчитанному ранее значению accuracy. 

И в последнюю очередь посмотрим на логистическую регрессию.

In [22]:
model = LogisticRegression(random_state=12345)# инициализируйте модель логистической регрессии с параметром random_state=12345
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
predictions_test = model.predict(features_test)
print(classification_report(target_test, predictions_test))

              precision    recall  f1-score   support

           0       0.69      0.98      0.81       440
           1       0.50      0.04      0.07       203

    accuracy                           0.68       643
   macro avg       0.59      0.51      0.44       643
weighted avg       0.63      0.68      0.58       643



Пожалуй данная модель работает хуже всего. Она только в половине случает верно пределяет тариф Ультра, как Ультра и выделила только 4% объектов имеющих значение Ультра.

**Вывод**

Модели построенные случайным лесом и решающим деревом ведут себя вполне адекватно при тех данных, которые есть, но также считаю, что можно было бы улучшить результат увеличив размер обучающей выборки. Сильный перевес в сторону данных по тару Смарт, дает о себе знать - модели лучши обучились и нашли более точные взаимосвязи. Второй вариант, наоборот уменьшить размер выборки, но так, чтобы соотношение данных по тарифу Ультра и Смарт было одинаковым, и посмотреть, что получится в этом случае. Хотя есть вероятность, что уменьшение выборки приведет к точу, что модель не сможет проследить хорошо связи между целевым признаком и признаками.

## Общий вывод

В данном проекте мы на основании данных о поведении клиентов, перешедших на тарифы Смарт и Ульта, пытаемся построить модель, которая будет предсказывать, какой тариф подойдет новым пользователям в зависимости от их предпочтений по количеству звонков, интернет-трафика и смс.

В первой части проекта мы посмотрели общие данные о файле. Данные оказались полностью пригодными для дальнейшей работы и делать предобработку нам не потребовалось.

Далее мы разбили данные на три выборки: обучающую (60% данных), валидационную, на которой проверяли различные модели (20% выборки) и тестовую (20% данных), которую использовали в самом конце, когда проверяли качество обученной модели, которую признали лучшей.

В третьем разделе мы рассмотрели три вида моделей: решающее дерево, случайный лес и логистическую регрессию. Поскольку нам нужно было решить задачу классификации, то взяли именно эти виды моделей. Постепенно меняя величину гиперпараметров, мы нашли такое сочетание, при котором модель выдает лучшую величину accuracy. Лучше всего на валидационной выборке вела себя модель случайный лес с количеством деревьев, равным 24 и глубиной деревьев 12. Именно эту модель мы и взяли для проверки на тестовой выборке.

В четвертом пункте, мы проверили выбранную модель на тестовой выборке. Accuracy (80,25% правильных ответов), оказался очень близким, к тому который получили на валидационной выборке (там было 80,55% правлильных ответов). Но насколько удовлетворит, такой результат заказчика сложно сказать.

Поскольку в выборке количество классов неравномерное, в пятом разделе мы попытались определить дополнительные метрики качества и проверить адекватность модели на тестовой выборке. Показатели precision (точность) для тарифа Смарт выше по всем трем моделям. Обучение по классу тариф Ультра прошло менее качественно. Хуже всего ведет себя логистическая регрессия. Считаю необходимым увеличить объем данных для обучения, причем добавить больше данных по тарифу Ультра.